In [1]:
import os.path
from os import path
import ipywidgets as widgets
import yaml
from ipytree import Tree, Node
from ipywidgets import interact, interactive, fixed, interact_manual, GridspecLayout, Layout, HBox, VBox
import dateutil.parser
from importlib import import_module


dir_workspace = os.path.join(os.getcwd(),'workspace_QWB_Development')
OBSERVABLES_DICT = dict()
GSO_STATIC = dict()
#DEFAULT_FETCHER_MODULE_NAME = "gs.qwb.functions.utils.OptionsDataFetcher"
#DEFAULT_OPTIONS_FETCHER_MODULE = import_module(DEFAULT_FETCHER_MODULE_NAME)

In [9]:
%%html
<style>
.lbl_bg{
    width:auto;
    background-color:yellow;
}
.box_style{
    width:100%;
    border : 2px solid black;
    height: auto;
    background-color:lightgray;
}
</style>

In [3]:
def form_function_parameter_display(display_type=None, default_value=None, display_name=None, default_options=None, disabled=False, default_rows_displayed=None):
    if display_type == "TEXT_AREA":
        w = widgets.Textarea(placeholder=default_value)
    elif display_type == "TEXT":
        w = widgets.Text(value=default_value, disabled=disabled)
    elif display_type == "DROPDOWN":
        w = widgets.Dropdown(options=default_options, disabled=disabled, value=None)
    elif display_type == "MULTIPLE_SELECT":
        w = widgets.SelectMultiple(options=default_options, disabled=disabled, rows=default_rows_displayed)
    elif display_type == "FLOAT_TEXT":
        w = widgets.FloatText(disabled=disabled, value=default_value, steps=0.001)
    elif display_type == "DATE_TIME":
        if default_value:
            #default_value = datetime.datetime.fromisoformat(default_value)
            default_value = dateutil.parser.isoparse(default_value)
        w = widgets.DatePicker(disabled=disabled, value=default_value)
    elif display_type == "RADIO":
        w = widgets.RadioButtons(options=default_options, disabled=False, value=default_value)
    w.layout.width = "auto"
    w.layout.left = "auto"
    w.layout.margin = '5px'
    return w

In [4]:
def visible_with_roles(item=None, current_roles=list()):
  assigned_roles_for_items = item.get('roles')
  
  if "admin" in current_roles:
    return True
  
  if not assigned_roles_for_items:
    return True
  
  for role in assigned_roles_for_items:
    if role in current_roles:
      return True
  
  return False

def fetch_default_options(options_value=None, widget=None):
    global GSO_STATIC
    if isinstance(options_value, str) and "GS_OPTIONS" in options_value:
        input_args = dict()
        input_args['value'] = options_value
        input_args['GSO_STATIC'] = GSO_STATIC
        #(GSO_STATIC, options_value) = DEFAULT_OPTIONS_FETCHER_MODULE.main(input_args)
        #print (GSO_STATIC)
      
    return options_value

def fetch_default_rows_displayed(options_value=None):
    if not options_value:
        return 20
    return options_value

def on_execute(button_obj):
    pass

In [5]:
def display_heading(categories=None):
    image_file = open(os.path.join(dir_workspace,'qwb_logo.png'), "rb")
    image = image_file.read()
    image_widget = widgets.Image(value=image, format='png', width='20%',)
    image_widget.layout.object_fit = "scale-down"
    
    vbox_children = list()
    vbox_children.append(image_widget)
    
    hbox_heading = HBox()
    hbox_heading.children = vbox_children
    
    hbox_heading.layout.justify_content = "center"
    return hbox_heading

In [6]:
def display_from_config(categories=None, current_role=None):
    i = 0
    tabs = widgets.Tab()
    tabs_children = list()
    
    for data in categories:
        if not visible_with_roles(item=data, current_roles=current_role):
          continue
        tabs.set_title(i, data['display_name'])

        accordian_widget = widgets.Accordion()
        accordian_widget.selected_index = None
        accordian_widget.add_class("box_style")
        accordian_children = list()
           
        j = 0
        for item in data['items']:
            accordian_widget.set_title(j, item['display_name'])
           
            if not 'input_parameters' in item or item['input_parameters'] == "":
                accordian_children.append(widgets.Label('No parameters defined'))
            else:
                #label_column_width = int(grid_size*15/100)
                #value_column_width = int(grid_size*30/100)

                output = widgets.Output(msg_id=str(i) + ":" + str(j))
                output.layout.height = 'auto'
                output.layout.width = '50%'
                output.layout.border = '1px'
                input_ref_dict = dict()

                vbox_widget = VBox()
                vbox_widget_children = list()
                
                for input_param in item.get('input_parameters'):
                    label_widget = widgets.Label(input_param['display_name'])
                    label_widget.layout.width = '40%'
                    label_widget.layout.margin = '5px'
                    disabled = False
                    if input_param.get('disabled'):
                      disabled = True
                    #print (input_param.get('display_name'))
                    default_options = fetch_default_options(options_value=input_param.get('default_options'))
                    default_rows_displayed = fetch_default_rows_displayed(options_value=input_param.get('default_rows_displayed'))
                    input_widget = form_function_parameter_display(display_name=input_param.get('display_name'),
                                                                   default_value=input_param.get('default_value'), 
                                                                   display_type=input_param.get('display_type'), 
                                                                   default_options=default_options, 
                                                                   disabled=disabled, 
                                                                   default_rows_displayed=default_rows_displayed) #label_column_width+value_column_width
                    
                    input_widget.layout.width = '60%'
                    hbox_widget = HBox([label_widget, input_widget])
                    vbox_widget_children.append(hbox_widget)
                
                execute_button = widgets.Button(layout=Layout(height='30px', width='auto'), button_style='info', description='Execute')
                execute_button.OUTPUT = output
                execute_button.INPUT_REF = input_ref_dict
                execute_button.INVOKE_REF = item['invoke_ref']
                execute_button.on_click(on_execute)
                execute_button.layout.justify_item = 'flex-end'
                execute_button.layout.width = '140px'
                
                vbox_widget_children.append(execute_button)
                vbox_widget.children = vbox_widget_children
                vbox_widget.layout.width = '50%'
                vbox_widget.layout.border='solid 1px'
                vbox_widget.add_class("box_style")
                hbox_master_widget = HBox([vbox_widget, output])
                accordian_children.append(hbox_master_widget)

            j = j + 1
        
        accordian_widget.children = accordian_children
        tabs_children.append(accordian_widget)
        i = i + 1
    
    tabs.children = tabs_children
    return tabs

In [7]:
def render(current_role=None):
    configuration_yaml = yaml.full_load(open(os.path.join(dir_workspace,'qwb_configuration.yaml')))

    
    box_widget_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='100%'
                    )
    box_widget_layout.background = "grey"
    box_widget = widgets.Box(layout=box_widget_layout)
    box_children = list()
    
    data_import = widgets.HTML(value='<div style="font-size:20px; text-align: left;vertical-align: middle; padding="10">GoldenSource Data Import</div>')
    data_import.layout.width = '100%'
    data_import.layout.padding = "10"
    
    data_import_label = widgets.HTML(value='<div style="font-size:20px; text-align: left;vertical-align: middle;"> Data Import </div>')
    data_import_label.layout.width = '100%'
    
    analytics_label = widgets.HTML(value='<div style="font-size:20px; text-align: left;vertical-align: middle;"> Functions </div>')
    analytics_label.layout.width = '100%'
    
    export_label = widgets.HTML(value='<div style="font-size:20px; text-align: left;vertical-align: middle;"> Export </div>')
    export_label.layout.width = '100%'
    
    data_extraction_tabs = display_from_config(configuration_yaml['data_extraction'], current_role=current_role)
    
    box_children.append(display_heading(configuration_yaml))
    box_children.append(data_import_label)
    box_children.append(data_extraction_tabs)
    box_children.append(analytics_label)
    #box_children.append(analytical_functions_tabs)
    box_children.append(export_label)
    #box_children.append(export_tabs)
    
    box_widget.children = box_children
    
    display(box_widget)

In [8]:
roles = ["admin"]
render(current_role=roles)

Box(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x1c\x00\x00\x00A\x0…